In [1]:
import sys
sys.path.append(r"D:\ig_pipeline")

In [2]:
from b1k_pipeline.utils import PipelineFS, get_targets

In [3]:
pipeline_fs = PipelineFS()

In [4]:
import json
from tqdm.notebook import tqdm
from fs.zipfs import ZipFS

problematic_targets = []
for tgt in tqdm(get_targets("combined")):
    target_fs = pipeline_fs.target_output(tgt)
    assert target_fs.exists("object_list.json")
    mesh_list = json.loads(target_fs.readtext("object_list.json"))["meshes"]
    
    if not target_fs.exists("meshes.zip"):
        problematic_targets.append(tgt)
        print(tgt, "missing meshes file")
        continue
        
    missing_meshes = []
    with target_fs.open("meshes.zip", "rb") as zip_file, ZipFS(zip_file) as zip_fs:
        for mesh in mesh_list:
            if not zip_fs.exists(mesh):
                missing_meshes.append(mesh)

    if missing_meshes:
        problematic_targets.append(tgt)

  0%|          | 0/1974 [00:00<?, ?it/s]

scenes/restaurant_urban missing meshes file


In [5]:
len(problematic_targets)

1

In [6]:
problematic_targets

['scenes/restaurant_urban']

In [7]:
print("dvc repro -s -f", " ".join(f"export_meshes@{tgt}" for tgt in problematic_targets))

dvc repro -s -f export_meshes@scenes/restaurant_urban
